In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib ipympl

import ImageD11.sinograms.point_by_point
import ImageD11.sinograms.dataset
import ImageD11.sinograms.properties
import ImageD11.columnfile

import ImageD11.nbGui.nb_utils as utils

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)

sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# This assumes you've only just segmented, so we need to add these things to the dataset:

# USER: specify the path to the parameter file

par_file = os.path.join(processed_data_root_dir, 'Si_refined.par')

# add them to the dataset

ds.parfile = par_file
ds.save()

In [ ]:
# Import the 2D columnfile, or make it if it doesn't exist

if os.path.exists(ds.col2dfile):
    cf_2d = ds.get_cf_2d_from_disk()
    cf_2d.parameters.loadparameters(ds.parfile)
    cf_2d.updateGeometry()
else:
    cf_2d = ds.get_cf_2d()

    # save the 2D peaks to file so we don't have to spatially correct them again
    ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)

In [ ]:
# filter the columnfile to discard weak peaks

cf_2d.filter(cf_2d.Number_of_pixels > 10)

In [ ]:
pbp_object = ImageD11.sinograms.point_by_point.PBP(ds.parfile,
                                                    ds,
                                                    hkl_tol=0.025,
                                                    fpks=0.9,
                                                    ds_tol=0.005,
                                                    etacut=0.1,
                                                    ifrac=5e-3,
                                                    cosine_tol=np.cos(np.radians(90 - ds.ostep)),
                                                    y0=0.0,
                                                    symmetry="cubic",
                                                    foridx=[0, 1, 2, 3, 4, 5, 6, 7],
                                                    forgen=[0, 1, 4],
                                                    uniqcut=0.85)

pbp_object.setpeaks(cf_2d)

In [ ]:
fig, ax = pbp_object.iplot()

In [ ]:
use_cluster = True

if use_cluster:
    bash_script_path = utils.prepare_pbp_bash(pbp_object, PYTHONPATH, 5)
    utils.slurm_submit_and_wait(bash_script_path, 15)
else:
    pbp_object.point_by_point(ds.pbpfile, loglevel=3)

In [ ]:
ds.save()

In [ ]:
if 1:
    raise ValueError("Change the 1 above to 0 to allow 'Run all cells' in the notebook")

In [ ]:
# Now that we're happy with our indexing parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_-50um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]
    
samples_dict = utils.find_datasets_to_process(ds.dataroot, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_200um"]}
    
# now we have our samples_dict, we can process our data:

first_pbp_object = pbp_object

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(ds.analysisroot, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        if not os.path.exists(dset_path):
            print(f"Missing DataSet file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        if os.path.exists(ds.pbpfile):
            print(f"Already have PBP file for {dataset} in sample {sample}, skipping")
            continue
        
        ds.parfile = par_file
        ds.save()
        
        if os.path.exists(ds.col2dfile):
            cf_2d = ds.get_cf_2d_from_disk()
            cf_2d.parameters.loadparameters(ds.parfile)
            cf_2d.updateGeometry()
        else:
            cf_2d = ds.get_cf_2d()

            # save the 2D peaks to file so we don't have to spatially correct them again
            ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)
            
        cf_2d.filter(cf_2d.Number_of_pixels > 10)
        
        pbp_object = ImageD11.sinograms.point_by_point.PBP(ds.parfile,
                                                            ds,
                                                            hkl_tol=first_pbp_object.hkl_tol,
                                                            fpks=first_pbp_object.fpks,
                                                            ds_tol=first_pbp_object.ds_tol,
                                                            etacut=first_pbp_object.etacut,
                                                            ifrac=first_pbp_object.ifrac,
                                                            cosine_tol=first_pbp_object.cosine_tol,
                                                            y0=first_pbp_object.y0,
                                                            symmetry=first_pbp_object.symmetry,
                                                            foridx=first_pbp_object.foridx,
                                                            forgen=first_pbp_object.forgen,
                                                            uniqcut=first_pbp_object.uniqcut)
        
        pbp_object.setpeaks(cf_2d)

        if use_cluster:
            bash_script_path = utils.prepare_pbp_bash(pbp_object, PYTHONPATH, 5)
            utils.slurm_submit_and_wait(bash_script_path, 15)
        else:
            pbp_object.point_by_point(ds.pbpfile, loglevel=3)
        
        ds.save()

print("Done!")